In [1]:
import numpy as np
import pandas as pd
import os
from data.buses import LTA
from data_proc import *

In [2]:
apiKey:str = "75cstockTLWN9wEcc9RE6Q=="

def to_df(data):
    data = pd.DataFrame.from_dict(data['value'])
    return data

lta = LTA(apiKey)



In [3]:
# Get df_bus_route
bus_route = lta.get_bus_routes()
df_bus_route = to_df(bus_route)
df_bus_route

,ServiceNo,Operator,Direction,StopSequence,BusStopCode,Distance,WD_FirstBus,WD_LastBus,SAT_FirstBus,SAT_LastBus,SUN_FirstBus,SUN_LastBus
0,10,SBST,1,1,75009,0.0,0500,2300,0500,2300,0500,2300
1,10,SBST,1,2,76059,0.6,0502,2302,0502,2302,0502,2302
2,10,SBST,1,3,76069,1.1,0504,2304,0504,2304,0503,2304
3,10,SBST,1,4,96289,2.3,0508,2308,0508,2309,0507,2308
4,10,SBST,1,5,96109,2.7,0509,2310,0509,2311,0508,2309
...,...,...,...,...,...,...,...,...,...,...,...,...
495,105,SBST,1,12,52189,5.0,0604,0018,0604,0018,0617,0016
496,105,SBST,1,13,52079,6.1,0607,0020,0606,0020,0620,0018
497,105,SBST,1,14,51099,7.3,0610,0023,0609,0023,0623,0020
498,105,SBST,1,15,40249,8.2,0612,0025,0611,0025,0625,0022


In [4]:
# preprocess_df_bus_route_1d: function to preprocess df_bus_route
# Input
df = df_bus_route

# Get ServiceNo with two directions
service_no_with_two_directions = df.groupby('ServiceNo')['Direction'].max()
service_no_with_two_directions = service_no_with_two_directions[service_no_with_two_directions == 2].index

# Filter the original DataFrame to include only ServiceNo with two directions
df = df[df['ServiceNo'].isin(service_no_with_two_directions)]

# Filter to include only Direction == 1 for each unique ServiceNo
df = df[df['Direction'] == 1] 

# Get the bus stop code when StopSequence is 1 for each bus
origin_bus_stop = df[df['StopSequence'] == 1].groupby('ServiceNo').agg(
    origin_bus_stop=('BusStopCode', 'first')
)
# Get the bus stop code when StopSequence is the max for each bus
destination_bus_stop = df.groupby('ServiceNo').agg(
    dest_bus_stop=('BusStopCode', 'last'),
    origin_dest_distance=('Distance', 'last')
)

# Merge origin_bus_stop into destination_bus_stop DataFrame
df_bus_route_processed = destination_bus_stop.merge(origin_bus_stop, left_index=True, right_index=True, how='left')

# Add Direction column
df_bus_route_processed['Direction'] = 1

# Reset index to make ServiceNo a column instead of index
df_bus_route_processed.reset_index(inplace=True)

# Reorder columns
df_bus_route_processed = df_bus_route_processed[['ServiceNo', 'Direction', 'origin_bus_stop', 'dest_bus_stop', 'origin_dest_distance']]

df_bus_route_processed


,ServiceNo,Direction,origin_bus_stop,dest_bus_stop,origin_dest_distance
0,10,1,75009,16009,31.6
1,100,1,66009,11009,24.7
2,103,1,66009,59009,17.9


In [5]:
# preprocess_df_bus_route_2d: function to preprocess df_bus_route
# Input
df = df_bus_route

# Get ServiceNo with two directions
service_no_with_two_directions = df.groupby('ServiceNo')['Direction'].max()
service_no_with_two_directions = service_no_with_two_directions[service_no_with_two_directions == 2].index

# Filter the original DataFrame to include only ServiceNo with two directions
df = df[df['ServiceNo'].isin(service_no_with_two_directions)]

# Filter to include only Direction == 1 for each unique ServiceNo
df1 = df[df['Direction'] == 1]
df2 = df[df['Direction'] == 2]  

def process_df_by_serviceNo(df):
    # Get the bus stop code when StopSequence is 1 for each bus
    origin_bus_stop = df[df['StopSequence'] == 1].groupby('ServiceNo').agg(
        origin_bus_stop=('BusStopCode', 'first')
    )
    # Get the bus stop code when StopSequence is the max for each bus
    destination_bus_stop = df.groupby('ServiceNo').agg(
        dest_bus_stop=('BusStopCode', 'last'),
        origin_dest_distance=('Distance', 'last')
    )

    # Merge origin_bus_stop into destination_bus_stop DataFrame
    df_bus_route_processed = destination_bus_stop.merge(origin_bus_stop, left_index=True, right_index=True, how='left')

    # Add Direction column
    df_bus_route_processed['Direction'] = df['Direction'].unique()[0]

    # Reset index to make ServiceNo a column instead of index
    df_bus_route_processed.reset_index(inplace=True)

    # Reorder columns
    df_bus_route_processed = df_bus_route_processed[['ServiceNo', 'Direction', 'origin_bus_stop', 'dest_bus_stop', 'origin_dest_distance']]
    return df_bus_route_processed

df1_processed = process_df_by_serviceNo(df1)
df2_processed = process_df_by_serviceNo(df2)

# Concatenate the two DataFrames
concatenated_df = pd.concat([df1_processed, df2_processed])

# Sort the concatenated DataFrame by ServiceNo and Direction
sorted_df = concatenated_df.sort_values(by=['ServiceNo', 'Direction'])

# Reset the index
sorted_df.reset_index(drop=True, inplace=True)

print(df1_processed)
print(df2_processed)
print(sorted_df)


  ServiceNo  Direction origin_bus_stop dest_bus_stop  origin_dest_distance
0        10          1           75009         16009                  31.6
1       100          1           66009         11009                  24.7
2       103          1           66009         59009                  17.9
  ServiceNo  Direction origin_bus_stop dest_bus_stop  origin_dest_distance
0        10          2           16009         75009                  31.9
1       100          2           11009         66009                  23.3
2       103          2           59009         66009                  17.8
  ServiceNo  Direction origin_bus_stop dest_bus_stop  origin_dest_distance
0        10          1           75009         16009                  31.6
1        10          2           16009         75009                  31.9
2       100          1           66009         11009                  24.7
3       100          2           11009         66009                  23.3
4       103          1   

In [6]:
# Test preprocess_df_bus_route_1d
df_1d = preprocess_df_bus_route_1d(df_bus_route)
df_1d

,ServiceNo,Direction,origin_bus_stop,dest_bus_stop,origin_dest_distance
0,10,1,75009,16009,31.6
1,100,1,66009,11009,24.7
2,103,1,66009,59009,17.9


In [7]:
# Test preprocess_df_bus_route_2d
df_2d = preprocess_df_bus_route_2d(df_bus_route)
df_2d

,ServiceNo,Direction,origin_bus_stop,dest_bus_stop,origin_dest_distance
0,10,1,75009,16009,31.6
1,10,2,16009,75009,31.9
2,100,1,66009,11009,24.7
3,100,2,11009,66009,23.3
4,103,1,66009,59009,17.9
5,103,2,59009,66009,17.8


In [8]:
# Each 'ServiceNo' -> min-max 'StopSequence' -> min-max 'Distance' 
df_bus_route[df_bus_route['ServiceNo'] == '10']


,ServiceNo,Operator,Direction,StopSequence,BusStopCode,Distance,WD_FirstBus,WD_LastBus,SAT_FirstBus,SAT_LastBus,SUN_FirstBus,SUN_LastBus
0,10,SBST,1,1,75009,0.0,0500,2300,0500,2300,0500,2300
1,10,SBST,1,2,76059,0.6,0502,2302,0502,2302,0502,2302
2,10,SBST,1,3,76069,1.1,0504,2304,0504,2304,0503,2304
3,10,SBST,1,4,96289,2.3,0508,2308,0508,2309,0507,2308
4,10,SBST,1,5,96109,2.7,0509,2310,0509,2311,0508,2309
...,...,...,...,...,...,...,...,...,...,...,...,...
143,10,SBST,2,70,96101,29.1,0658,0112,0721,0113,0730,0108
144,10,SBST,2,71,96281,29.5,0659,0113,0722,0114,0732,0109
145,10,SBST,2,72,76061,30.8,0705,0117,0726,0117,0736,0113
146,10,SBST,2,73,76051,31.3,0707,0118,0728,0119,0737,0114


In [9]:
# 'Direction' == 1 and 'Direction' == 2 have different origin-destination distances, can take maximum of these too
    # Better to pick only 'Direction' == 1 since it reduces frontend work when plotting the bus stops
df_bus_route[(df_bus_route['ServiceNo'] == '10') & (df_bus_route['Direction'] == 1)]

,ServiceNo,Operator,Direction,StopSequence,BusStopCode,Distance,WD_FirstBus,WD_LastBus,SAT_FirstBus,SAT_LastBus,SUN_FirstBus,SUN_LastBus
0,10,SBST,1,1,75009,0.0,0500,2300,0500,2300,0500,2300
1,10,SBST,1,2,76059,0.6,0502,2302,0502,2302,0502,2302
2,10,SBST,1,3,76069,1.1,0504,2304,0504,2304,0503,2304
3,10,SBST,1,4,96289,2.3,0508,2308,0508,2309,0507,2308
4,10,SBST,1,5,96109,2.7,0509,2310,0509,2311,0508,2309
...,...,...,...,...,...,...,...,...,...,...,...,...
69,10,SBST,1,70,16051,30.4,0623,0021,0618,0025,0617,0021
70,10,SBST,1,71,16061,30.6,0624,0021,0618,0026,0617,0021
71,10,SBST,1,72,16071,30.8,0625,0022,0619,0027,0618,0022
72,10,SBST,1,73,16081,31.1,0626,0023,0620,0028,0619,0023


In [10]:
# start: 16009, end: 75009
df_bus_route[(df_bus_route['ServiceNo'] == '10') & (df_bus_route['Direction'] == 2)]

,ServiceNo,Operator,Direction,StopSequence,BusStopCode,Distance,WD_FirstBus,WD_LastBus,SAT_FirstBus,SAT_LastBus,SUN_FirstBus,SUN_LastBus
74,10,SBST,2,1,16009,0.0,0600,2400,0600,2400,0615,2400
75,10,SBST,2,2,16089,0.4,0602,0002,0602,0001,0617,0001
76,10,SBST,2,3,16079,0.7,0603,0003,0603,0002,0618,0002
77,10,SBST,2,4,16069,1.0,0605,0004,0605,0003,0620,0003
78,10,SBST,2,5,16059,1.3,0606,0005,0606,0004,0621,0004
...,...,...,...,...,...,...,...,...,...,...,...,...
143,10,SBST,2,70,96101,29.1,0658,0112,0721,0113,0730,0108
144,10,SBST,2,71,96281,29.5,0659,0113,0722,0114,0732,0109
145,10,SBST,2,72,76061,30.8,0705,0117,0726,0117,0736,0113
146,10,SBST,2,73,76051,31.3,0707,0118,0728,0119,0737,0114


In [11]:
# Loop service bus route?
    # Conclusion: only cares about 'Direction'=1, and max'Distance' from this direction 
df_bus_route[df_bus_route['ServiceNo'] == '101']

,ServiceNo,Operator,Direction,StopSequence,BusStopCode,Distance,WD_FirstBus,WD_LastBus,SAT_FirstBus,SAT_LastBus,SUN_FirstBus,SUN_LastBus
272,101,SBST,1,1,66009,0.0,0515,2400,0515,2400,0515,2400
273,101,SBST,1,2,62131,0.5,0517,0002,0517,0002,0517,0002
274,101,SBST,1,3,62141,0.9,0518,0003,0518,0003,0518,0003
275,101,SBST,1,4,63011,1.4,0520,0005,0520,0005,0520,0005
276,101,SBST,1,5,63021,1.9,0521,0007,0521,0007,0521,0007
277,101,SBST,1,6,63031,2.5,0515,0009,0515,0009,0515,0009
278,101,SBST,1,7,63041,2.8,0516,0010,0516,0010,0516,0010
279,101,SBST,1,8,63051,3.0,0517,0011,0517,0011,0517,0011
280,101,SBST,1,9,63061,3.3,0518,0012,0518,0013,0518,0012
281,101,SBST,1,10,64011,3.5,0519,0013,0519,0014,0519,0013


In [12]:
# Bus 100
df_bus_route[(df_bus_route['ServiceNo'] == '100') & (df_bus_route['Direction'] == 2)]

,ServiceNo,Operator,Direction,StopSequence,BusStopCode,Distance,WD_FirstBus,WD_LastBus,SAT_FirstBus,SAT_LastBus,SUN_FirstBus,SUN_LastBus
205,100,SBST,2,1,11009,0.0,0600,2400,0600,2400,0600,2400
206,100,SBST,2,2,11359,0.2,0601,0001,0601,0001,0601,0001
207,100,SBST,2,3,11189,0.6,0603,0003,0603,0003,0603,0003
208,100,SBST,2,4,11179,0.8,0604,0004,0604,0004,0604,0005
209,100,SBST,2,5,11169,1.4,0607,0007,0607,0007,0606,0008
210,100,SBST,2,6,11059,2.0,0609,0009,0609,0009,0608,0010
211,100,SBST,2,7,11049,2.4,0610,0010,0610,0010,0609,0011
212,100,SBST,2,8,11039,2.8,0612,0011,0611,0011,0610,0013
213,100,SBST,2,9,11029,3.2,0613,0012,0612,0011,0610,0014
214,100,SBST,2,10,11519,3.8,0615,0014,0614,0013,0612,0016


In [13]:
# Get df_bus_stop

In [14]:
# Load existing zipped data
current_folder_path = os.getcwd()
data_folder_name = "data"
data_file_name = "origin_destination_bus_202402.zip"
data_file_path = os.path.join(current_folder_path, data_folder_name, data_file_name)
df = pd.read_csv(data_file_path)
df

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS
0,2024-02,WEEKDAY,16,BUS,4168,10051,4
1,2024-02,WEEKENDS/HOLIDAY,16,BUS,4168,10051,3
2,2024-02,WEEKDAY,14,BUS,80119,90079,12
3,2024-02,WEEKENDS/HOLIDAY,14,BUS,80119,90079,2
4,2024-02,WEEKDAY,17,BUS,44069,17229,5
...,...,...,...,...,...,...,...
5659682,2024-02,WEEKENDS/HOLIDAY,10,BUS,77009,5049,3
5659683,2024-02,WEEKDAY,10,BUS,77009,5049,2
5659684,2024-02,WEEKDAY,17,BUS,28661,42071,1
5659685,2024-02,WEEKENDS/HOLIDAY,15,BUS,80159,11161,2


In [15]:
# Preprocess total-trips df to get monthly origin-dest total trips
# Filter 'PT_TYPE' == 'BUS'
df = df[df['PT_TYPE'] == 'BUS']

# Group by YEAR_MONTH, ORIGIN_PT_CODE, DESTINATION_PT_CODE and sum TOTAL_TRIPS
condensed_df = df.groupby(['YEAR_MONTH', 'ORIGIN_PT_CODE', 'DESTINATION_PT_CODE']).agg({'TOTAL_TRIPS': 'sum'}).reset_index()

# Display the condensed DataFrame
print(condensed_df)

       YEAR_MONTH  ORIGIN_PT_CODE  DESTINATION_PT_CODE  TOTAL_TRIPS
0         2024-02            1012                 1112         2417
1         2024-02            1012                 1113         1907
2         2024-02            1012                 1121         1567
3         2024-02            1012                 1211         1490
4         2024-02            1012                 1311         1851
...           ...             ...                  ...          ...
334881    2024-02           99189                98139           21
334882    2024-02           99189                99019            1
334883    2024-02           99189                99029           14
334884    2024-02           99189                99039           26
334885    2024-02           99189                99049           62

[334886 rows x 4 columns]


In [16]:
# Test proprocess_totalTrips_df
df_total_trips = preprocess_totalTrips_df(df)
df_total_trips

,YEAR_MONTH,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS
0,2024-02,1012,1112,2417
1,2024-02,1012,1113,1907
2,2024-02,1012,1121,1567
3,2024-02,1012,1211,1490
4,2024-02,1012,1311,1851
...,...,...,...,...
334881,2024-02,99189,98139,21
334882,2024-02,99189,99019,1
334883,2024-02,99189,99029,14
334884,2024-02,99189,99039,26


In [17]:
# df_2d
df_2d

,ServiceNo,Direction,origin_bus_stop,dest_bus_stop,origin_dest_distance
0,10,1,75009,16009,31.6
1,10,2,16009,75009,31.9
2,100,1,66009,11009,24.7
3,100,2,11009,66009,23.3
4,103,1,66009,59009,17.9
5,103,2,59009,66009,17.8


In [18]:
print(sorted_df.dtypes)
print(df_total_trips.dtypes)

ServiceNo                object
Direction                 int64
origin_bus_stop          object
dest_bus_stop            object
origin_dest_distance    float64
dtype: object
YEAR_MONTH             object
ORIGIN_PT_CODE          int64
DESTINATION_PT_CODE     int64
TOTAL_TRIPS             int64
dtype: object


In [19]:
# Merge df_total_trips into df_2d
merged_df = df_2d.merge(df_total_trips, 
                             left_on=['origin_bus_stop', 'dest_bus_stop'], 
                             right_on=['ORIGIN_PT_CODE', 'DESTINATION_PT_CODE'], 
                             how='left')
merged_df

,ServiceNo,Direction,origin_bus_stop,dest_bus_stop,origin_dest_distance,YEAR_MONTH,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS
0,10,1,75009,16009,31.6,2024-02,75009,16009,75
1,10,2,16009,75009,31.9,2024-02,16009,75009,60
2,100,1,66009,11009,24.7,2024-02,66009,11009,15
3,100,2,11009,66009,23.3,2024-02,11009,66009,86
4,103,1,66009,59009,17.9,2024-02,66009,59009,677
5,103,2,59009,66009,17.8,2024-02,59009,66009,520


In [20]:
# Test merge_distance_totalTrips
df_distance_totalTrips = merge_distance_totalTrips(df_2d, df_total_trips)
df_distance_totalTrips

,ServiceNo,Direction,origin_bus_stop,dest_bus_stop,origin_dest_distance,YEAR_MONTH,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS
0,10,1,75009,16009,31.6,2024-02,75009,16009,75
1,10,2,16009,75009,31.9,2024-02,16009,75009,60
2,100,1,66009,11009,24.7,2024-02,66009,11009,15
3,100,2,11009,66009,23.3,2024-02,11009,66009,86
4,103,1,66009,59009,17.9,2024-02,66009,59009,677
5,103,2,59009,66009,17.8,2024-02,59009,66009,520


In [21]:
# df_bus_stops
bus_stops = lta.get_bus_stops()
bus_stops = bus_stops['value']
nums =[500* i for i in range(1,11)]

for num in nums:
    bus_stops2 = lta.get_bus_stops2(num)  # modified method to generate dummy  * the api can call 500 record at once, so need to call multiple times to get all records
    bus_stops2 = bus_stops2['value']
    bus_stops.extend(bus_stops2)

print(len(bus_stops))
df_bus_stops =  pd.DataFrame.from_dict(bus_stops)
df_bus_stops

http://datamall2.mytransport.sg/ltaodataservice/BusStops?$skip=500
http://datamall2.mytransport.sg/ltaodataservice/BusStops?$skip=1000
http://datamall2.mytransport.sg/ltaodataservice/BusStops?$skip=1500
http://datamall2.mytransport.sg/ltaodataservice/BusStops?$skip=2000
http://datamall2.mytransport.sg/ltaodataservice/BusStops?$skip=2500
http://datamall2.mytransport.sg/ltaodataservice/BusStops?$skip=3000
http://datamall2.mytransport.sg/ltaodataservice/BusStops?$skip=3500
http://datamall2.mytransport.sg/ltaodataservice/BusStops?$skip=4000
http://datamall2.mytransport.sg/ltaodataservice/BusStops?$skip=4500
http://datamall2.mytransport.sg/ltaodataservice/BusStops?$skip=5000
5109


,BusStopCode,RoadName,Description,Latitude,Longitude
0,01012,Victoria St,Hotel Grand Pacific,1.296848,103.852536
1,01013,Victoria St,St. Joseph's Ch,1.297710,103.853225
2,01019,Victoria St,Bras Basah Cplx,1.296990,103.853022
3,01029,Nth Bridge Rd,Opp Natl Lib,1.296673,103.854414
4,01039,Nth Bridge Rd,Bugis Cube,1.298208,103.855491
...,...,...,...,...,...
5104,99139,Changi Village Rd,Blk 5,1.388195,103.987234
5105,99161,Nicoll Dr,Bef Changi Beach CP 3,1.390262,103.992957
5106,99171,Nicoll Dr,Changi Beach CP 2,1.391128,103.991021
5107,99181,Telok Paku Rd,Bef S'pore Aviation Ac,1.387754,103.988503


In [22]:
# df_taps
# filepath = 'transport_node_bus_202402.zip'
# taps = lta.get_passenger_vol_by_bus_stops(filepath)
# df_taps = to_df(taps)

# Path to your CSV file
csv_file_path = 'data/transport_node_bus_202402.csv'

# Read the CSV file into a DataFrame
df_taps = pd.read_csv(csv_file_path)

# Display the first few rows of the DataFrame
df_taps

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,PT_CODE,TOTAL_TAP_IN_VOLUME,TOTAL_TAP_OUT_VOLUME
0,2024-02,WEEKENDS/HOLIDAY,17.0,BUS,45379,190,120
1,2024-02,WEEKDAY,17.0,BUS,45379,442,470
2,2024-02,WEEKDAY,13.0,BUS,80051,2873,2873
3,2024-02,WEEKENDS/HOLIDAY,13.0,BUS,80051,1473,1403
4,2024-02,WEEKDAY,13.0,BUS,5319,109,540
...,...,...,...,...,...,...,...
198898,2024-02,WEEKDAY,7.0,BUS,51011,1851,1870
198899,2024-02,WEEKENDS/HOLIDAY,18.0,BUS,70149,391,273
198900,2024-02,WEEKDAY,18.0,BUS,70149,2088,1547
198901,2024-02,WEEKENDS/HOLIDAY,6.0,BUS,13139,37,119


In [23]:
# Preprocess df_taps
df = df_taps

# Filter 'PT_TYPE' == 'BUS'
df = df[df['PT_TYPE'] == 'BUS']

# Group by YEAR_MONTH, PT_CODE, and sum TOTAL_TAP_IN_VOLUME TOTAL_TAP_OUT_VOLUME
condensed_df = df.groupby(['YEAR_MONTH', 'PT_CODE']).agg({'TOTAL_TAP_IN_VOLUME': 'sum', 'TOTAL_TAP_OUT_VOLUME': 'sum'}).reset_index()
condensed_df['TOTAL_TAP_VOLUME'] = condensed_df['TOTAL_TAP_IN_VOLUME'] + condensed_df['TOTAL_TAP_OUT_VOLUME']

# Display the condensed DataFrame
condensed_df

,YEAR_MONTH,PT_CODE,TOTAL_TAP_IN_VOLUME,TOTAL_TAP_OUT_VOLUME,TOTAL_TAP_VOLUME
0,2024-02,1012,28041,38209,66250
1,2024-02,1013,22187,16891,39078
2,2024-02,1019,16367,25547,41914
3,2024-02,1029,34387,22084,56471
4,2024-02,1039,58100,50405,108505
...,...,...,...,...,...
5089,2024-02,99139,36,19201,19237
5090,2024-02,99161,146,894,1040
5091,2024-02,99171,799,4034,4833
5092,2024-02,99181,13473,13863,27336


In [24]:
# Test preprocess_df_taps
df_taps = preprocess_df_taps(df_taps)
df_taps

,YEAR_MONTH,PT_CODE,TOTAL_TAP_IN_VOLUME,TOTAL_TAP_OUT_VOLUME,TOTAL_TAP_VOLUME
0,2024-02,1012,28041,38209,66250
1,2024-02,1013,22187,16891,39078
2,2024-02,1019,16367,25547,41914
3,2024-02,1029,34387,22084,56471
4,2024-02,1039,58100,50405,108505
...,...,...,...,...,...
5089,2024-02,99139,36,19201,19237
5090,2024-02,99161,146,894,1040
5091,2024-02,99171,799,4034,4833
5092,2024-02,99181,13473,13863,27336


In [25]:
# Check very large tap number
df_taps[df_taps['PT_CODE'] == 75009]

,YEAR_MONTH,PT_CODE,TOTAL_TAP_IN_VOLUME,TOTAL_TAP_OUT_VOLUME,TOTAL_TAP_VOLUME
4137,2024-02,75009,1177882,1177024,2354906


In [26]:
# Merge df_taps in df_distance_totalTrips

# Merge the dataframes based on matching YEAR_MONTH and PT_CODE
merged_df = df_distance_totalTrips.merge(df_taps, 
                                         left_on=['YEAR_MONTH', 'ORIGIN_PT_CODE'], 
                                         right_on=['YEAR_MONTH', 'PT_CODE'], 
                                         how='left')

# Merge again for the destination PT_CODE
merged_df = merged_df.merge(df_taps, 
                             left_on=['YEAR_MONTH', 'DESTINATION_PT_CODE'], 
                             right_on=['YEAR_MONTH', 'PT_CODE'], 
                             suffixes=('_origin', '_destination'), 
                             how='left')

# Calculate the passenger volume by summing TOTAL_TAP_VOLUME from both origin and destination
merged_df['passenger_volume'] = merged_df['TOTAL_TAP_VOLUME_origin'] + merged_df['TOTAL_TAP_VOLUME_destination']

# Calculate the passenger volume by summing TOTAL_TAP_VOLUME from both origin and destination
merged_df['passenger_volume'] = merged_df['TOTAL_TAP_VOLUME_origin'] + merged_df['TOTAL_TAP_VOLUME_destination']

# Drop unnecessary columns
merged_df.drop(columns=['PT_CODE_origin', 'TOTAL_TAP_IN_VOLUME_origin', 'TOTAL_TAP_OUT_VOLUME_origin',  'PT_CODE_destination',
                        'TOTAL_TAP_IN_VOLUME_destination', 'TOTAL_TAP_OUT_VOLUME_destination', 'ORIGIN_PT_CODE', 'DESTINATION_PT_CODE'], inplace=True)

merged_df

,ServiceNo,Direction,origin_bus_stop,dest_bus_stop,origin_dest_distance,YEAR_MONTH,TOTAL_TRIPS,TOTAL_TAP_VOLUME_origin,TOTAL_TAP_VOLUME_destination,passenger_volume
0,10,1,75009,16009,31.6,2024-02,75,2354906,62057,2416963
1,10,2,16009,75009,31.9,2024-02,60,62057,2354906,2416963
2,100,1,66009,11009,24.7,2024-02,15,470695,53894,524589
3,100,2,11009,66009,23.3,2024-02,86,53894,470695,524589
4,103,1,66009,59009,17.9,2024-02,677,470695,1599822,2070517
5,103,2,59009,66009,17.8,2024-02,520,1599822,470695,2070517


In [27]:
# Test merge_taps_distance_totalTrips
df_taps_distance_totalTrips = merge_taps_distance_totalTrips(df_taps, df_distance_totalTrips)
df_taps_distance_totalTrips

,ServiceNo,Direction,origin_bus_stop,dest_bus_stop,origin_dest_distance,YEAR_MONTH,TOTAL_TRIPS,TOTAL_TAP_VOLUME_origin,TOTAL_TAP_VOLUME_destination,passenger_volume
0,10,1,75009,16009,31.6,2024-02,75,2354906,62057,2416963
1,10,2,16009,75009,31.9,2024-02,60,62057,2354906,2416963
2,100,1,66009,11009,24.7,2024-02,15,470695,53894,524589
3,100,2,11009,66009,23.3,2024-02,86,53894,470695,524589
4,103,1,66009,59009,17.9,2024-02,677,470695,1599822,2070517
5,103,2,59009,66009,17.8,2024-02,520,1599822,470695,2070517


In [28]:
# Compute bus/car CO2 emission
df = df_taps_distance_totalTrips
bus_CO2_rate = 0.48  # kg/km
car_CO2_rate = 0.167  # kg/km
bus2car_ratio = 1/4  # approximation: 1/4 passengers taking car and carpool
df['co2_by_bus'] = df['origin_dest_distance']*df['TOTAL_TRIPS']*bus_CO2_rate
df['co2_by_car'] = df['origin_dest_distance']*df['TOTAL_TRIPS']*df['passenger_volume']*bus2car_ratio*car_CO2_rate
df['co2_reduction'] = df['co2_by_car'] - df['co2_by_bus']
df

,ServiceNo,Direction,origin_bus_stop,dest_bus_stop,origin_dest_distance,YEAR_MONTH,TOTAL_TRIPS,TOTAL_TAP_VOLUME_origin,TOTAL_TAP_VOLUME_destination,passenger_volume,co2_by_bus,co2_by_car,co2_reduction
0,10,1,75009,16009,31.6,2024-02,75,2354906,62057,2416963,1137.600,2.391524e+08,2.391513e+08
1,10,2,16009,75009,31.9,2024-02,60,62057,2354906,2416963,918.720,1.931383e+08,1.931374e+08
2,100,1,66009,11009,24.7,2024-02,15,470695,53894,524589,177.840,8.114539e+06,8.114362e+06
3,100,2,11009,66009,23.3,2024-02,86,53894,470695,524589,961.824,4.388641e+07,4.388545e+07
4,103,1,66009,59009,17.9,2024-02,677,470695,1599822,2070517,5816.784,1.047555e+09,1.047550e+09
5,103,2,59009,66009,17.8,2024-02,520,1599822,470695,2070517,4442.880,8.001264e+08,8.001220e+08


In [31]:
# Test df_co2
df_co2 = get_df_co2(df_taps_distance_totalTrips)
df_co2

,ServiceNo,Direction,origin_bus_stop,dest_bus_stop,origin_dest_distance,YEAR_MONTH,TOTAL_TRIPS,TOTAL_TAP_VOLUME_origin,TOTAL_TAP_VOLUME_destination,passenger_volume,co2_by_bus,co2_by_car,co2_reduction
0,10,1,75009,16009,31.6,2024-02,75,2354906,62057,2416963,1137.600,2.391524e+08,2.391513e+08
1,10,2,16009,75009,31.9,2024-02,60,62057,2354906,2416963,918.720,1.931383e+08,1.931374e+08
2,100,1,66009,11009,24.7,2024-02,15,470695,53894,524589,177.840,8.114539e+06,8.114362e+06
3,100,2,11009,66009,23.3,2024-02,86,53894,470695,524589,961.824,4.388641e+07,4.388545e+07
4,103,1,66009,59009,17.9,2024-02,677,470695,1599822,2070517,5816.784,1.047555e+09,1.047550e+09
5,103,2,59009,66009,17.8,2024-02,520,1599822,470695,2070517,4442.880,8.001264e+08,8.001220e+08


## Checking df_OD_volume

In [32]:
# Load existing zipped data
current_folder_path = os.getcwd()
data_folder_name = "data"
data_file_name = "origin_destination_bus_202402.zip"
data_file_path = os.path.join(current_folder_path, data_folder_name, data_file_name)
df = pd.read_csv(data_file_path)
df

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS
0,2024-02,WEEKDAY,16,BUS,4168,10051,4
1,2024-02,WEEKENDS/HOLIDAY,16,BUS,4168,10051,3
2,2024-02,WEEKDAY,14,BUS,80119,90079,12
3,2024-02,WEEKENDS/HOLIDAY,14,BUS,80119,90079,2
4,2024-02,WEEKDAY,17,BUS,44069,17229,5
...,...,...,...,...,...,...,...
5659682,2024-02,WEEKENDS/HOLIDAY,10,BUS,77009,5049,3
5659683,2024-02,WEEKDAY,10,BUS,77009,5049,2
5659684,2024-02,WEEKDAY,17,BUS,28661,42071,1
5659685,2024-02,WEEKENDS/HOLIDAY,15,BUS,80159,11161,2


In [33]:
# Bus 10
df[(df['ORIGIN_PT_CODE'] == 75009) & (df['DESTINATION_PT_CODE'] == 76051)]

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS
338370,2024-02,WEEKDAY,20,BUS,75009,76051,2
338371,2024-02,WEEKENDS/HOLIDAY,20,BUS,75009,76051,2
695264,2024-02,WEEKENDS/HOLIDAY,14,BUS,75009,76051,2
695267,2024-02,WEEKDAY,14,BUS,75009,76051,4
1222117,2024-02,WEEKDAY,13,BUS,75009,76051,9
1222120,2024-02,WEEKENDS/HOLIDAY,13,BUS,75009,76051,1
1241832,2024-02,WEEKDAY,8,BUS,75009,76051,1
1241833,2024-02,WEEKENDS/HOLIDAY,8,BUS,75009,76051,1
1604870,2024-02,WEEKDAY,19,BUS,75009,76051,1
2454098,2024-02,WEEKDAY,12,BUS,75009,76051,6


In [34]:
# Find rows where ORIGIN_PT_CODE equals DESTINATION_PT_CODE
    # Findings: No origin-destination total trips for loop service
df[df['ORIGIN_PT_CODE'] == df['DESTINATION_PT_CODE']]

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS


In [35]:
# Bus 101: loop service
# May need to get rid of loop-service bus (ex: 101) on the map, since there's no origin-destination total-trips for this bus

origin = 66009
destination = 62189
df[(df['ORIGIN_PT_CODE'] == origin) & (df['DESTINATION_PT_CODE'] == destination)]
# df[(df['ORIGIN_PT_CODE'] == origin)]

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS
1090788,2024-02,WEEKDAY,14,BUS,66009,62189,1
1924406,2024-02,WEEKDAY,17,BUS,66009,62189,2
2459734,2024-02,WEEKENDS/HOLIDAY,21,BUS,66009,62189,1
3103159,2024-02,WEEKENDS/HOLIDAY,16,BUS,66009,62189,2
3103161,2024-02,WEEKDAY,16,BUS,66009,62189,1
3568708,2024-02,WEEKENDS/HOLIDAY,12,BUS,66009,62189,1
3700910,2024-02,WEEKENDS/HOLIDAY,11,BUS,66009,62189,1
4459307,2024-02,WEEKDAY,20,BUS,66009,62189,2
4797179,2024-02,WEEKENDS/HOLIDAY,10,BUS,66009,62189,1


In [36]:
# Bus 100
# No origin-destination total_trips data for this route?

origin = 11009
destination = 66009
df[(df['ORIGIN_PT_CODE'] == origin) & (df['DESTINATION_PT_CODE'] == destination)]
# df[(df['ORIGIN_PT_CODE'] == origin)]

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS
295840,2024-02,WEEKDAY,12,BUS,11009,66009,3
401694,2024-02,WEEKDAY,19,BUS,11009,66009,5
401695,2024-02,WEEKENDS/HOLIDAY,19,BUS,11009,66009,1
907393,2024-02,WEEKENDS/HOLIDAY,21,BUS,11009,66009,6
907394,2024-02,WEEKDAY,21,BUS,11009,66009,1
1198918,2024-02,WEEKENDS/HOLIDAY,9,BUS,11009,66009,1
1318349,2024-02,WEEKENDS/HOLIDAY,16,BUS,11009,66009,1
1318350,2024-02,WEEKDAY,16,BUS,11009,66009,6
1724755,2024-02,WEEKDAY,14,BUS,11009,66009,10
1724756,2024-02,WEEKENDS/HOLIDAY,14,BUS,11009,66009,3


In [37]:
# Bus 100
# Direction 1 has same total trips as Direction 2?
    # Findings: seems not

origin = 66009
destination = 11009
df[(df['ORIGIN_PT_CODE'] == origin) & (df['DESTINATION_PT_CODE'] == destination)]
# df[(df['ORIGIN_PT_CODE'] == origin)]

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS
1318351,2024-02,WEEKDAY,16,BUS,66009,11009,1
1724757,2024-02,WEEKDAY,14,BUS,66009,11009,1
3997829,2024-02,WEEKDAY,6,BUS,66009,11009,13


In [38]:
# Bus 10

origin = 75009
destination = 16009
df[(df['ORIGIN_PT_CODE'] == origin) & (df['DESTINATION_PT_CODE'] == destination)]

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS
13929,2024-02,WEEKDAY,9,BUS,75009,16009,1
795985,2024-02,WEEKDAY,11,BUS,75009,16009,3
795987,2024-02,WEEKENDS/HOLIDAY,11,BUS,75009,16009,2
1111922,2024-02,WEEKDAY,22,BUS,75009,16009,1
1111924,2024-02,WEEKENDS/HOLIDAY,22,BUS,75009,16009,1
1187255,2024-02,WEEKENDS/HOLIDAY,7,BUS,75009,16009,1
1187256,2024-02,WEEKDAY,7,BUS,75009,16009,1
1625229,2024-02,WEEKDAY,13,BUS,75009,16009,8
1625230,2024-02,WEEKENDS/HOLIDAY,13,BUS,75009,16009,4
1805412,2024-02,WEEKDAY,6,BUS,75009,16009,1


In [39]:
origin = 16009
destination = 75009
df[(df['ORIGIN_PT_CODE'] == origin) & (df['DESTINATION_PT_CODE'] == destination)]

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS
13928,2024-02,WEEKDAY,9,BUS,16009,75009,2
795986,2024-02,WEEKDAY,11,BUS,16009,75009,3
1111923,2024-02,WEEKENDS/HOLIDAY,22,BUS,16009,75009,1
1625228,2024-02,WEEKDAY,13,BUS,16009,75009,3
1859554,2024-02,WEEKDAY,19,BUS,16009,75009,1
2161372,2024-02,WEEKDAY,17,BUS,16009,75009,2
2161373,2024-02,WEEKENDS/HOLIDAY,17,BUS,16009,75009,1
2701474,2024-02,WEEKDAY,21,BUS,16009,75009,3
2963835,2024-02,WEEKENDS/HOLIDAY,15,BUS,16009,75009,1
2963836,2024-02,WEEKDAY,15,BUS,16009,75009,3
